<a href="https://colab.research.google.com/github/AI-Lai/ExperimentingWithAtari/blob/main/ATARI_TP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Dependencies

In [ ]:
%pip install tensorflow
!pip install pyglet
print("done")
!pip install gym keras-rl2 gym[atari]
print("done")
%pip install -U gym>=0.21.0
print("done")
%pip install -U gym[atari,accept-rom-license]
print("DONE")
#!pip install pyvirtualdisplay
#!pip install reverb

# 1. Test Environment in OpenAI Gym

Import gym and random: this will enable us to see what's happening inside the game.

Ideally the lines 

"!pip install gym keras-rl2 gym[atari]

%pip install -U gym>=0.21.0

%pip install -U gym[atari,accept-rom-license]" 

allow us to run any atari gym environment without preinstalling anything.


In [2]:
#todo
game= "gamename"
env = gym.make(game)


/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


We can try to run some episodes to see what's happening in the game.
For the moment we can just run random actions.

In order to visualize the game, you can call **env.render()**, possibly in mode "human". At the end of each event print the results.

Note: render does not work on colab, try it in your own notebook.

In [4]:
episodes = 3
for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
      pass
      
    print('Episode:{} Score:{}'.format(episode+1, score))
env.close()

Episode:1 Score:50.0
Episode:2 Score:105.0
Episode:3 Score:55.0


# 2. Create a Deep Learning Model with Keras

Import the different libraries (numpy, keras.models.Sequential(), the different layers, and Adam for the optimisation)

In [5]:
import numpy as np
# import sequential, Dense, Flatten, Convolution2D, adam


Build a CNN. The structure can be adapted, but we can use as a starting point the structure used in https://www.nature.com/articles/nature14236/ and https://arxiv.org/abs/1511.06581

Since there are some sparse bugs, wrap the model in a function build_model just in case of necessity.

In [28]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    ###### layers
    model.add(Dense(actions, activation='linear'))
    return model
# del model
# in case of random error, rebuild the model
model = build_model(height, width, channels, actions)

As usual, before startin to mess up everything, we can peek a look at our network.

If it seems too massive for your architechture you can prune it, just for the sake of seeing it working

In [ ]:
model.summary()

# 3. Build Agent with Keras-RL

Ok, now the interesting part.

Using classical Q-Learning/SARSA is not feasible, can you tell why?

Indeed, we will use the CNN built in the previous paragraph to "suggest" to the agent the best action at each step.


Here we import from keras the libraries to build and train such an agent.

In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy


We can build our agent.
We specify a policy and a memory. We can also set up a dueling network as explained in  https://arxiv.org/abs/1511.06581

In [17]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

We can now fit our dqn on our environment.
We should specify the number of steps: for this kind of problem we need among 1M and 10M steps in order to reach pseudo-human (or even super-human) level.

Nevertheless, it can be a good exercice to try out on 10K steps. If everything worked fine it will take more or less half an hour, the time for you to take a coffee and enjoy the different papers proposed in the TP.



Lastly, it is very funny to set the parameter visualize to True, in order to watch what is effectively learning. Note that this operation will make your fit function way slower (unbearably slower).

In [25]:
#???? fit(env, steps, )

Ideally, we can now test the performance of our agent.
Again, we can print to screen the AI playing.

In [ ]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history['episode_reward']))

# 4. Reloading Agent from Memory

If you got here you realised a very bad Atari player, good job.

The objective can be to download the weights calculated by some other nice ppl on the web.


In [20]:
dqn.save_weights('bad_dqn_weights.h5f')

In [26]:
del model, dqn
#rebuild model and dqn after this

In [27]:
dqn.load_weights('good_dqn_weights.h5f')

Up to you.
What could we do to improve performance?
Here are some suggestions and explications to get better results.
https://towardsdatascience.com/tutorial-double-deep-q-learning-with-dueling-network-architectures-4c1b3fb7f756


https://medium.com/analytics-vidhya/building-a-powerful-dqn-in-tensorflow-2-0-explanation-tutorial-d48ea8f3177a

https://github.com/EvolvedSquid/tutorials/tree/master/dqn

https://www.freecodecamp.org/news/an-introduction-to-deep-q-learning-lets-play-doom-54d02d8017d8/

(the last one is quite outdated, now, but still presents some interesting points)

Try to apply the suggestions !